In [2]:
import pandas as pd

def read_file(filepath: str):
    return pd.read_json(filepath, orient="records")

df = read_file("data/obfuscated_data_06.json")

In [6]:
import ast

with open('data/test_indices_2.txt', 'r') as file:
    test_indices_2 = text_indices_2 = ast.literal_eval(file.read().strip())

print(len(test_indices_2))

13613


In [8]:
df.iloc[0]

full_text              VISUALIZATION\n\nChallenge:\n\nAs a student (o...
document                                                               0
tokens                 [VISUALIZATION, \n\n, Challenge, :, \n\n, As, ...
trailing_whitespace    [False, False, False, False, False, True, True...
labels                 [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
Name: 0, dtype: object

In [15]:
num_char = 0
for i in range (len(df)):
    if df.iloc[i].document in test_indices_2:
        num_char += len(df.iloc[i].full_text)
print(num_char)

50661491


In [4]:
total_tokens = sum(len(df.iloc[i].tokens) for i in range(len(df)))

print(f"Total number of tokens: {total_tokens}")

Total number of tokens: 16746176


In [16]:
total_tokens_test = sum(len(df.iloc[i].tokens) for i in range(len(df)) if df.iloc[i].document in test_indices_2)

print(f"Total number of tokens in test set: {total_tokens_test}")

Total number of tokens in test set: 10051912


In [3]:
total_tokens = sum(len(df.iloc[i].tokens) for i in range(len(df)))

print(f"Total number of tokens: {total_tokens}")

Total number of tokens: 16746176


In [7]:
# Cost per token in dollars (GPT-4o)
input_cost_per_token_gpt4o = 5 / 1000000  # Cost per input token
output_cost_per_token_gpt4o = 15 / 1000000  # Cost per output token

# Cost per token in dollars (GPT-4o-mini)
input_cost_per_token_gpt4omini = 0.15 / 1000000  # Cost per input token
output_cost_per_token_gpt4omini = 0.6 / 1000000  # Cost per output token

# Calculate the total cost
total_cost_gpt4o = total_tokens * (input_cost_per_token_gpt4o + output_cost_per_token_gpt4o)
total_cost_gpt4omini = total_tokens * (input_cost_per_token_gpt4omini + output_cost_per_token_gpt4omini)

print(f"Estimated total cost (GPT-4o): ${total_cost_gpt4o:.2f}")
print(f"Estimated total cost (GPT-4o-mini): ${total_cost_gpt4omini:.2f}")

Estimated total cost (GPT-4o): $334.92
Estimated total cost (GPT-4o-mini): $12.56


**Consider Azure AI as baseline**
Model performance first
Efficiency (average time to process one transcript and show the length of each transcript),
cost ()

In [12]:
# Estimate Fine-tuning cost
import json
import tiktoken  # This library is for counting tokens, make sure you have it installed

# Function to count tokens in a text using tiktoken
def count_tokens(text, model="gpt-4o"):
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

# Calculate total tokens in JSONL file
def calculate_total_tokens_and_cost(jsonl_file_path, base_cost_per_million_tokens, epochs=1):
    total_tokens = 0

    with open(jsonl_file_path, 'r') as file:
        for line in file:
            data = json.loads(line.strip())
            
            # Convert the conversation messages to a single string for token counting
            conversation_text = " ".join([message['content'] for message in data['messages']])
            
            # Count the number of tokens for each conversation
            total_tokens += count_tokens(conversation_text)

    # Calculate the estimated cost
    estimated_cost = (base_cost_per_million_tokens / 1000000) * total_tokens * epochs

    return total_tokens, estimated_cost

# Constants for GPT-4o and GPT-4o-mini
base_cost_per_million_tokens_gpt4o = 25
base_cost_per_million_tokens_gpt4o_mini = 3

# Path to your JSONL file
jsonl_file_path = 'output/pii_detected_all.jsonl'  # Replace with your actual file path
jsonl_file_path_train = 'output/pii_detected_train.jsonl'  # Replace with your actual file path

# Calculate tokens and costs for GPT-4o
tokens_gpt4o, cost_gpt4o = calculate_total_tokens_and_cost(jsonl_file_path, base_cost_per_million_tokens_gpt4o)
tokens_gpt4o_mini, cost_gpt4o_mini = calculate_total_tokens_and_cost(jsonl_file_path, base_cost_per_million_tokens_gpt4o_mini)

tokens_gpt4o_train, cost_gpt4o_train = calculate_total_tokens_and_cost(jsonl_file_path_train, base_cost_per_million_tokens_gpt4o)
tokens_gpt4o_mini_train, cost_gpt4o_mini_train = calculate_total_tokens_and_cost(jsonl_file_path_train, base_cost_per_million_tokens_gpt4o_mini)

print(f"Total tokens: {tokens_gpt4o}")
print(f"Estimated cost for GPT-4o: ${cost_gpt4o:.2f}")
print(f"Estimated cost for GPT-4o-mini: ${cost_gpt4o_mini:.2f}\n")

print(f"Total tokens (25% training): {tokens_gpt4o_train}")
print(f"Estimated cost for GPT-4o (25% training): ${cost_gpt4o_train:.2f}")
print(f"Estimated cost for GPT-4o-mini (25% training): ${cost_gpt4o_mini_train:.2f}")

Total tokens: 1380147
Estimated cost for GPT-4o: $34.50
Estimated cost for GPT-4o-mini: $4.14

Total tokens (25% training): 343543
Estimated cost for GPT-4o (25% training): $8.59
Estimated cost for GPT-4o-mini (25% training): $1.03


In [4]:
8634067/1000000 * 3

25.902200999999998

In [5]:
6634067/1000000 * 3

19.902200999999998